In [6]:
import pandas as pd
import scanpy as sc
from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_rand_score
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score, fowlkes_mallows_score, silhouette_score
from utils import sankey_plot

In [7]:
DIR = './data/'
DATASET_NAMES = ['PBMC1', 'PBMC3', 'PBMC4']#, 'PBMC2', 
TOOLS = ['monocle', 'scanpy', 'scvi-tools', 'seurat', 'COTAN']
PARAMS_TUNING = ['default', 'celltypist', 'antibody']

In [8]:
def compute_scores(dir, dataset, labels_df, labels_matched, ground_truth_labels):
    scores = {}
    scores['NMI'] = {}
    scores['ARI'] = {}
    scores['homogeneity'] = {}
    scores['completeness'] = {}
    scores['v_measure'] = {}
    scores['fowlkes_mallows'] = {}
    for tool in TOOLS:
        scores['NMI'][tool] = normalized_mutual_info_score(labels_pred=labels_df['cluster_'+tool], labels_true=labels_df[f'cluster_{ground_truth_labels}'], average_method='arithmetic')
        scores['ARI'][tool] = adjusted_rand_score(labels_pred=labels_df['cluster_'+tool], labels_true=labels_df[f'cluster_{ground_truth_labels}'])
        scores['homogeneity'][tool] = homogeneity_score(labels_pred=labels_df['cluster_'+tool], labels_true=labels_df[f'cluster_{ground_truth_labels}'])
        scores['completeness'][tool] = completeness_score(labels_pred=labels_df['cluster_'+tool], labels_true=labels_df[f'cluster_{ground_truth_labels}'])
        scores['v_measure'][tool] = v_measure_score(labels_pred=labels_df['cluster_'+tool], labels_true=labels_df[f'cluster_{ground_truth_labels}'])
        scores['fowlkes_mallows'][tool] = fowlkes_mallows_score(labels_pred=labels_df['cluster_'+tool], labels_true=labels_df[f'cluster_{ground_truth_labels}'])
    scores_df = pd.DataFrame(scores)
    scores_df.to_csv(f'{dir}{dataset}/scores_{labels_matched}_{ground_truth_labels}.csv')
    scores_df.to_latex(f'{dir}{dataset}/scores_{labels_matched}_{ground_truth_labels}.tex')
    display(scores_df)

In [9]:
for tuning in PARAMS_TUNING:
    for dataset in DATASET_NAMES:
        print('------------------------------')
        print(f'{dataset} - matching {tuning} labels' if tuning != 'default' else f'{dataset} - default labels')
        
        # concat tools labels
        labels_df = pd.read_csv(f'{DIR}{dataset}/COTAN/{tuning}/clustering_labels.csv', index_col=0)
        labels_df.rename(columns={"cluster": "cluster_COTAN"}, inplace=True)
        for tool in [t for t in TOOLS if t != 'COTAN']:
            tool_labels_df = pd.read_csv(f'{DIR}{dataset}/{tool}/{tuning}/clustering_labels.csv', index_col=0)
            labels_df = labels_df.merge(tool_labels_df, how='inner', on='cell')
            labels_df.rename(columns={"cluster": f"cluster_{tool}"}, inplace=True)
        
        # load and concat celltypist labels
        celltypist_df = pd.read_csv(f'{DIR}{dataset}/celltypist/celltypist_labels.csv', index_col=0)
        celltypist_df.index = celltypist_df.index.str[:-2]
        celltypist_df = labels_df.merge(celltypist_df, how='inner', on='cell')
        celltypist_df.rename(columns={"cluster.ids": f"cluster_celltypist"}, inplace=True)
        celltypist_mapping_df = pd.read_csv(f'{DIR}{dataset}/celltypist/celltypist_mapping.csv', index_col=0)
        
        # load and concat protein surface labels
        antibody_df = pd.read_csv(f'{DIR}{dataset}/antibody_annotation/antibody_labels.csv', index_col=0)
        antibody_df = labels_df.merge(antibody_df, how='inner', on='cell')
        antibody_df.rename(columns={"cluster.ids": f"cluster_antibody"}, inplace=True)
        antibody_mapping_df = pd.read_csv(f'{DIR}{dataset}/antibody_annotation/antibody_mapping.csv', index_col=1)

        # read dataset
        adata = sc.read_10x_mtx(
            f'{DIR}{dataset}/filtered/10X/',
            var_names='gene_symbols',
            cache=False
        )
        # keep only labelled cells
        adata.var_names_make_unique()
        subset_cells = adata.obs_names.isin(labels_df.index)
        adata = adata[subset_cells, :]

        # compute silhouette score
        silhouette = {}
        for tool in TOOLS:
            silhouette[tool] = silhouette_score(adata.X, labels_df[f'cluster_{tool}'])
        if tuning=='celltypist':
            silhouette['celltypist'] = silhouette_score(adata.X, celltypist_df[f'cluster_celltypist'])
        elif tuning=='antibody':
            silhouette['antibody'] = silhouette_score(adata.X, antibody_df[f'cluster_antibody'])
        silhouette_df = pd.DataFrame(silhouette, index=[0])
        silhouette_df.to_csv(f'{DIR}{dataset}/{tuning}_silhouette.csv')
        silhouette_df.to_latex(f'{DIR}{dataset}/{tuning}_silhouette.tex')

        # compute scores comparing each tool labels with celltypist labels
        if tuning == 'celltypist' or tuning == 'default':
            compute_scores(DIR, dataset, celltypist_df, tuning, 'celltypist')
            labels = []
            labels_titles = []
            for tool in TOOLS:
                labels.append(celltypist_df[f'cluster_{tool}'].to_list())
                labels_titles.append(tool)
            labels.append(celltypist_df[f'cluster_celltypist'].map(celltypist_mapping_df['go'].to_dict()).to_list())
            labels_titles.append('celltypist')
            title = f'{dataset} - matching {tuning} labels' if tuning != 'default' else f'{dataset} - default labels'
            sankey_plot(labels=labels, labels_titles=labels_titles, title=title, path=f'{DIR}{dataset}/{tuning}_celltypist.html')
        
        # compute scores comparing each tool labels with protein labels
        if tuning == 'antibody' or tuning == 'default':
            compute_scores(DIR, dataset, antibody_df, tuning, 'antibody')
            labels = []
            labels_titles = []
            for tool in TOOLS:
                labels.append(antibody_df[f'cluster_{tool}'].to_list())
                labels_titles.append(tool)
            labels.append(antibody_df[f'cluster_antibody'].map(antibody_mapping_df['go'].to_dict()).to_list())
            labels_titles.append('antibody')
            title = f'{dataset} - matching {tuning} labels' if tuning != 'default' else f'{dataset} - default labels'
            sankey_plot(labels=labels, labels_titles=labels_titles, title=title, path=f'{DIR}{dataset}/{tuning}_antibody.html')

------------------------------
PBMC1 - default labels


/Users/irenetesta/anaconda3/envs/chl/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning:

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead



,NMI,ARI,homogeneity,completeness,v_measure,fowlkes_mallows
monocle,0.586477,0.434154,0.419100,0.976441,0.586477,0.641586
scanpy,0.707991,0.363700,0.821675,0.621941,0.707991,0.483365
scvi-tools,0.759984,0.540722,0.803366,0.721048,0.759984,0.626231
seurat,0.770234,0.588503,0.770632,0.769836,0.770234,0.661907
COTAN,0.760531,0.573026,0.824545,0.705741,0.760531,0.653886


,NMI,ARI,homogeneity,completeness,v_measure,fowlkes_mallows
monocle,0.611988,0.425929,0.446299,0.973344,0.611988,0.635502
scanpy,0.659645,0.391203,0.795577,0.563386,0.659645,0.507730
scvi-tools,0.708581,0.551051,0.776228,0.651780,0.708581,0.632750
seurat,0.738344,0.643097,0.764146,0.714228,0.738344,0.706018
COTAN,0.711422,0.579912,0.799933,0.640547,0.711422,0.657343


------------------------------
PBMC3 - default labels


...

/Users/irenetesta/anaconda3/envs/chl/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning:

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead



,NMI,ARI,homogeneity,completeness,v_measure,fowlkes_mallows
monocle,0.633333,0.503352,0.609373,0.659256,0.633333,0.567586
scanpy,0.737439,0.695008,0.736412,0.738468,0.737439,0.732379
scvi-tools,0.712286,0.638310,0.692394,0.733355,0.712286,0.684119
seurat,0.719240,0.547590,0.797156,0.655200,0.719240,0.613912
COTAN,0.710483,0.621643,0.674240,0.750844,0.710483,0.682058


------------------------------
PBMC4 - matching antibody labels


/Users/irenetesta/anaconda3/envs/chl/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning:

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead



,NMI,ARI,homogeneity,completeness,v_measure,fowlkes_mallows
monocle,0.646496,0.476298,0.583962,0.724030,0.646496,0.552480
scanpy,0.684939,0.495584,0.645785,0.729147,0.684939,0.560952
scvi-tools,0.677494,0.471424,0.630909,0.731506,0.677494,0.540577
seurat,0.724976,0.590682,0.616860,0.879046,0.724976,0.688576
COTAN,0.629502,0.410868,0.575363,0.694887,0.629502,0.494384
